# Dear Jupyter Notebook Reader

In [3]:
# import all necessary packages  
import pandas as pd 
import numpy as np
import os 
import seaborn as sns
from src import helpers
from src import Textprocessor as tp

In [4]:
train = helpers.read_df('data/train.csv', index_col='listing_id')
#test = pd.read_csv('data/test.csv', index_col='listing_id')
#reviews = pd.read_csv('data/reviews.csv', index_col='listing_id')

In [9]:
processor = Textprocessor.TextProcessor()

In [17]:
# missing values
train = helpers.change_data_types(train)
test = helpers.change_data_types(test)


In [20]:
reviews.head(5)

,reviewer_id,comments,review_id
listing_id,,,
8Z94Z3WMCO,93896,"The flat was bright, comfortable and clean and...",30672
8Z94Z3WMCO,97890,We stayed with Adriano and Valerio for a week ...,32236
8Z94Z3WMCO,104133,Adriano was a fantastic host. We felt very at ...,41044
8Z94Z3WMCO,122714,We had a most wonderful stay with Adriano and ...,48926
8Z94Z3WMCO,111543,I'm not sure which of us misunderstood the s...,58352


In [7]:
pUrl_list = train.picture_url.unique().tolist()
filenames = train.index.unique().tolist()

In [31]:
pictures = train.picture_url

In [40]:
pictures.index[0]

'9Q1RD6H7IO'

In [26]:
pictures.index.iloc

Index(['9Q1RD6H7IO', '6XDPZPGLSR', 'SND9OPN6IY', 'SE3B7BXN9G', 'E0C82VK0VF',
       '3OP12CBRJ5', 'MJJG3JOEYC', 'J0IOAQXJT0', 'SKWS3WE08F', 'HKAPWVENPH',
       ...
       '1YSHZXURHW', 'XMFL68O3MQ', 'SPCNFIJREV', 'ZHTXO3K31Z', 'OGM4XZ1K4W',
       'HSY40NEHOI', 'XZXB1NS63Y', 'Z2T8G6D82D', '8B1Q8UJWLH', 'BKSRYBXCFL'],
      dtype='object', name='listing_id', length=55284)

In [37]:
for i in range(0,len(pictures)):
    print(pictures.iloc[i], pictures.index[i] + '.png')


AttributeError: 'Index' object has no attribute 'iloc'

In [19]:
pUrl_list

257d40b.jpg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/0e433ce3-88c3-4b23-9f89-29ab6326d6d0.jpg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/b5e80ee9-254a-4b7a-b1be-33fc4ac29f89.jpg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/15189f48-8134-4b46-a660-51a15d7d0303.jpg?aki_policy=large',
 'https://a0.muscache.com/4ea/air/v2//pictures/dad22279-11b4-42cf-818c-2536b0c920b4.jpg?t=r:w1200-h720-sfit,e:fjpg-c85',
 'https://a0.muscache.com/im/pictures/0665a722-5c41-4bcc-a6ab-5da9ec34d19e.jpg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/e4b2ad15-6cb3-4cb7-8062-5746d0670f88.jpg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/18251191/2298cca4_original.jpg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/b10408ec-5827-4fdd-816c-1560677a4a9f.jpg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/4ee74fd8-4735-487e-bc7f-07ba05419ebe.jpg?aki_policy=large',
 'https://a0.muscache.com/im/pictures/a859c08f-c85f-4d2c-9c3b-9acbd384a544

In [13]:
train.loc[train.duplicated()==True]

,name,summary,space,description,experiences_offered,neighborhood_overview,transit,house_rules,picture_url,host_id,...,guests_included,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,cancellation_policy,reviews_per_month
listing_id,,,,,,,,,,,,,,,,,,,,,
